In [2]:

import panel
code = 'print(1)'

editor = panel.widgets.CodeEditor(
    value=code,
    language="python",
    theme="monokai",
    min_height=100,
    max_width=400,
    sizing_mode="stretch_width",
)
ipywidget_code_editor = panel.ipywidget(editor)
ipywidget_code_editor

BokehModel(combine_events=True, render_bundle={'docs_json': {'ef875638-da66-49dc-a8ff-ed02ecc2d85d': {'version…

# Test and implement new ideas for pyironFlow

In [3]:
import ipywidgets as widgets
from pygments import highlight
from pygments.lexers import get_lexer_by_name
from pygments.formatters import HtmlFormatter
from IPython.display import display, HTML

In [4]:
def update_highlighting(change):
    code = editor.value
    lexer = get_lexer_by_name("python", stripall=True)
    formatter = HtmlFormatter(style="default")
    highlighted = highlight(code, lexer, formatter)
    output.value = f"<style>{formatter.get_style_defs()}</style>{highlighted}"

In [5]:
editor = widgets.Textarea(
    value="# Enter your Python code here\n",
    placeholder="Type something",
    description="Editor:",
    disabled=False,
    layout={"width": "100%", "height": "300px"},
)

output = widgets.HTML(
    value="<pre># Enter your Python code here</pre>",
    placeholder="",
    description="Output:",
)

In [6]:
editor.observe(update_highlighting, names="value")
display(editor, output)

Textarea(value='# Enter your Python code here\n', description='Editor:', layout=Layout(height='300px', width='…

HTML(value='<pre># Enter your Python code here</pre>', description='Output:', placeholder='')

In [7]:
import panel
import param

# Enable the CodeEditor extension
panel.extension("codeeditor")

# Create a CodeEditor widget
code_editor = panel.widgets.CodeEditor(
    value="import numpy as np\n\ndef example_function():\n    return np.random.rand(10)",
    language="python",
    theme="monokai",
    height=300,
    sizing_mode="stretch_width",
)

# Display the code editor
code_editor

BokehModel(combine_events=True, render_bundle={'docs_json': {'ba808578-3bfa-48f3-a6d9-ad3f6ac56fb6': {'version…

In [8]:
out = widgets.Output()
with out:
    display(code_editor)

In [9]:
out

Output()

In [10]:
import sys
from pathlib import Path

# sys.path.remove('/Users/joerg/python_projects/git_libs/pyiron-xyflow')
# sys.path.remove('/Users/joerg/python_projects/git_libs/pyiron_workflow')
sys.path.insert(0, "/Users/neugebauer/git_libs/pyiron_nodes")
sys.path

['/Users/neugebauer/git_libs/pyiron_nodes',
 '/users/neugebauer/gitProjects/pyiron_workflows',
 '/Users/neugebauer/git_libs/pyironFlow',
 '/Users/neugebauer/mambaforge/envs/py12/lib/python312.zip',
 '/Users/neugebauer/mambaforge/envs/py12/lib/python3.12',
 '/Users/neugebauer/mambaforge/envs/py12/lib/python3.12/lib-dynload',
 '',
 '/Users/neugebauer/mambaforge/envs/py12/lib/python3.12/site-packages']

In [11]:
from pyiron_workflow import Workflow, as_function_node

In [12]:
from pyiron_workflow import Workflow

import pyiron_nodes as pn

wf = Workflow("compute_elastic_constants")
wf.engine = pn.atomistic.engine.ase.M3GNet()
wf.bulk = pn.atomistic.structure.build.Bulk("Pb", cubic=True)
wf.input_elastic = pn.atomistic.property.elastic.InputElasticTensor()
wf.elastic = pn.atomistic.property.elastic.ElasticConstants(
    structure=wf.bulk, engine=wf.engine, parameters=wf.input_elastic
)

2024-11-24 20:07:43,641 - pyiron_log - WARNING - pyiron found a 'dynamic' folder in the /Users/neugebauer/pyiron/resources resource directory. These are no longer supported in pyiron_base >=0.7.0. They are replaced by Project.create_job_class() and Project.wrap_python_function().
2024-11-24 20:07:43,642 - pyiron_log - WARNING - pyiron found a 'templates' folder in the /Users/neugebauer/pyiron/resources resource directory. These are no longer supported in pyiron_base >=0.7.0. They are replaced by Project.create_job_class() and Project.wrap_python_function().


In [13]:
from pyironflow.reactflow import get_edges as _get_edges
from pyironflow.reactflow import get_nodes as _get_nodes
from pyironflow.reactflow import get_node_from_path

In [14]:
def filter_and_flatten_nested_dict_keys(data, keys_to_keep):
    def filter_and_flatten_dict(d, keys):
        result = {}
        for key in keys:
            if "/" in key:
                top_key, nested_key = key.split("/", 1)
                if top_key in d and isinstance(d[top_key], dict):
                    result[f"{top_key}__{nested_key}"] = d[top_key].get(nested_key)
            else:
                if key in d:
                    result[key] = d[key]
        return result

    return [filter_and_flatten_dict(item, keys_to_keep) for item in data]

In [15]:
def rename_keys(dict_list, key_mapping):
    """
    Rename keys in a list of dictionaries.

    Args:
    dict_list (list): A list of dictionaries to modify.
    key_mapping (dict): A dictionary mapping old keys to new keys.

    Returns:
    list: A new list of dictionaries with renamed keys.
    """
    result = []
    for d in dict_list:
        new_dict = {}
        for old_key, value in d.items():
            new_key = key_mapping.get(old_key, old_key)
            new_dict[new_key] = value
        result.append(new_dict)
    return result

In [16]:
def different_indices(list1, list2):
    return [i for i in range(len(list1)) if list1[i] != list2[i]]

In [17]:
def nodes_from_dict(nodes_dict):
    return [(node["label"], node["import_path"]) for node in nodes_dict]


def edges_from_dict(edges_dict):
    edges = []
    for edge in edges_dict:
        source = edge["source"]
        source_handle = edge["sourceHandle"]
        if source == "":
            if isinstance(source_handle, str):
                source_handle = f"__str_{source_handle}"
        edges.append(
            (
                f'{edge["source"]}/{source_handle}',
                f'{edge["target"]}/{edge["targetHandle"]}',
            )
        )

    return edges


def get_nodes(wf, keys_to_keep=["data/label", "data/import_path"]):
    key_mapping = {"data__label": "label", "data__import_path": "import_path"}
    nodes_dict = filter_and_flatten_nested_dict_keys(_get_nodes(wf), keys_to_keep)
    nodes_dict = rename_keys(nodes_dict, key_mapping)
    return nodes_dict

In [18]:
import dataclasses


@dataclasses.dataclass
class WorkflowGraph:
    nodes: list = dataclasses.field(default_factory=list)
    edges: list = dataclasses.field(default_factory=list)

In [19]:
def get_graph(wf):
    # get edges between nodes
    keys_to_keep = ["target", "targetHandle", "source", "sourceHandle"]
    edges = filter_and_flatten_nested_dict_keys(_get_edges(wf), keys_to_keep)

    # add edges for non-default inputs
    nodes = get_nodes(
        wf,
        keys_to_keep=[
            "data/label",
            "data/import_path",
            "data/target_values",
            "data/target_labels",
        ],
    )
    nodes_non_default_inp_param = []
    for node in nodes:
        label = node["label"]
        import_path = node["import_path"]
        node_obj = get_node_from_path(import_path)()
        changed_args = different_indices(
            node_obj.inputs.to_list(), node["data__target_values"]
        )
        for i in changed_args:
            value = node["data__target_values"][i]
            handle = node["data__target_labels"][i]
            if value not in ("NonPrimitive", "NotData"):
                edge = dict(
                    target=label,
                    targetHandle=handle,
                    source="",
                    sourceHandle=value,
                )

                edges.append(edge)
                inp_node = dict(label=f"var_{label}__{handle}", data__import_path=value)
                nodes_non_default_inp_param.append(inp_node)

        nodes = get_nodes(
            wf,
            keys_to_keep=["data/label", "data/import_path"],
        )

    key_mapping = {"data__label": "label", "data__import_path": "import_path"}
    nodes = rename_keys(nodes_non_default_inp_param + nodes, key_mapping=key_mapping)

    graph = WorkflowGraph(nodes=nodes_from_dict(nodes), edges=edges_from_dict(edges))
    # graph = WorkflowGraph(nodes=nodes_from_dict(nodes), edges=edges)
    return graph


graph = get_graph(wf)
graph.edges, graph.nodes

([('engine/engine', 'elastic/engine'),
  ('bulk/structure', 'elastic/structure'),
  ('input_elastic/dataclass', 'elastic/parameters'),
  ('/__str_Pb', 'bulk/name'),
  ('/True', 'bulk/cubic')],
 [('var_bulk__name', 'Pb'),
  ('var_bulk__cubic', True),
  ('engine', 'pyiron_nodes.atomistic.engine.ase.M3GNet'),
  ('bulk', 'pyiron_nodes.atomistic.structure.build.Bulk'),
  ('input_elastic',
   'pyiron_nodes.atomistic.property.elastic.InputElasticTensor'),
  ('elastic', 'pyiron_nodes.atomistic.property.elastic.ElasticConstants')])

In [20]:
def generate_wf_code(wf, module_a="pyiron_workflow", module_b="pyiron_nodes"):
    """
    Generate Python source code based on input parameters.

    Args:
        label (str): The label to use in the generated code.
        module_a (str): The name of the module to import from.
        module_b (str): The name of the module to import.

    Returns:
        str: The generated Python source code.
    """
    import black

    code = f"""
from {module_a} import Workflow
import {module_b}

wf = Workflow('{wf.label}')

"""
    graph = get_graph(wf)

    # Add nodes to Workflow
    for node in graph.nodes:
        label, import_path = node

        if not label.startswith('var_'):
            code += f"""wf.{label} = {import_path}("""

            # Add edges
            first_edge = True
            for edge in graph.edges:
                edge_source, edge_target = edge
                target, target_handle = edge_target.split('/')
                source, source_handle = edge_source.split('/')
                if target == label:
                    if first_edge:
                        first_edge = False
                    else:
                        code += """, """
                    if source == '':
                        if source_handle.startswith('__str_'):
                            code += f"""{target_handle}='{source_handle[6:]}'"""
                        else:
                            code += f"""{target_handle}={source_handle}"""
                    else:
                        code += f"""{target_handle}=wf.{source}"""
            code += f""") \n"""

    formatted_code = black.format_str(code, mode=black.FileMode())

    return formatted_code

In [21]:
from pyiron_workflow import Workflow

import pyiron_nodes as pn

wf = Workflow("compute_elastic_constants")
wf.engine = pn.atomistic.engine.ase.M3GNet()
wf.bulk = pn.atomistic.structure.build.Bulk("Pb", cubic=True)
wf.input_elastic = pn.atomistic.property.elastic.InputElasticTensor()
wf.elastic = pn.atomistic.property.elastic.ElasticConstants(
    structure=wf.bulk, engine=wf.engine, parameters=wf.input_elastic
)

In [22]:
code = generate_wf_code(wf)

In [23]:
code_editor = panel.widgets.CodeEditor(
    value=code,
    language="python",
    theme="monokai",
    height=300,
    sizing_mode="stretch_width",
)

# Display the code editor
code_editor

BokehModel(combine_events=True, render_bundle={'docs_json': {'da43f149-d92c-4ddd-9846-9b91c2e8a590': {'version…

In [24]:
print(code_editor.value)

from pyiron_workflow import Workflow
import pyiron_nodes

wf = Workflow("compute_elastic_constants")

wf.engine = pyiron_nodes.atomistic.engine.ase.M3GNet()
wf.bulk = pyiron_nodes.atomistic.structure.build.Bulk(name="Pb", cubic=True)
wf.input_elastic = pyiron_nodes.atomistic.property.elastic.InputElasticTensor()
wf.elastic = pyiron_nodes.atomistic.property.elastic.ElasticConstants(
    engine=wf.engine, structure=wf.bulk, parameters=wf.input_elastic
)



In [25]:
def run_python_code(code): 
    import ast
    import io

    tree = ast.parse(code)
    with io.StringIO() as output:
        exec(compile(tree, filename="<string>", mode="exec"), globals(), locals())
        print(output.getvalue())
        return output.getvalue()  



In [26]:
out = run_python_code(code_editor.value)

In [27]:
out

''

In [30]:
import ast
import io

code = """
from pyiron_workflow import Workflow
import pyiron_nodes

wf = Workflow("compute_elastic_constants")

wf.engine = pyiron_nodes.atomistic.engine.ase.M3GNet()
wf.bulk = pyiron_nodes.atomistic.structure.build.Bulk(name="Pb", cubic=True)
wf.input_elastic = pyiron_nodes.atomistic.property.elastic.InputElasticTensor()
wf.elastic = pyiron_nodes.atomistic.property.elastic.ElasticConstants(
    engine=wf.engine, structure=wf.bulk, parameters=wf.input_elastic
)

print ('aaa')
"""

tree = ast.parse(code)
with io.StringIO() as output:
    exec(compile(tree, filename="<string>", mode="exec"), globals(), locals())
    print(output.getvalue())

# tree = ast.parse(code)
# exec(compile(tree, filename="<string>", mode="exec"))

aaa



In [31]:
import ast
import io
import contextlib

def execute_code(code):
    tree = ast.parse(code)
    with io.StringIO() as output:
        with contextlib.redirect_stdout(output):
            exec(compile(tree, filename="<string>", mode="exec"), globals(), locals())
        return output.getvalue()

out = execute_code(code_editor.value)
print(out)

In [32]:
out

''

In [33]:
import ipywidgets as widgets
from IPython.display import display

def create_editor_with_output():
    # Create an editor widget
    editor = widgets.Textarea(
        value='# Enter your Python code here\nprint("Hello, World!")',
        description='Code:',
        layout={'width': '100%', 'height': '200px'}
    )

    # Create an output widget
    output = widgets.Output(layout={'border': '1px solid black'})

    # Function to execute the code and display the output
    def run_code(button):
        with output:
            output.clear_output(wait=True)
            exec(code_editor.value)

    # Create a run button
    run_button = widgets.Button(description="Run Code")
    run_button.on_click(run_code)

    # Display the widgets
    display(code_editor, run_button, output)

# Call the function to create and display the widgets
create_editor_with_output()

BokehModel(combine_events=True, render_bundle={'docs_json': {'24f425fb-f8d5-41fb-93e6-4658f8245944': {'version…

Button(description='Run Code', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [34]:
import ipywidgets as widgets
from IPython.display import display


def toolbar(code, width=400):
    editor = panel.widgets.CodeEditor(
        value=code,
        language="python",
        theme="monokai",
        min_height=100,
        max_width=width,
        sizing_mode="stretch_width",
    )
    ipywidget_code_editor = panel.ipywidget(editor)

    button_layout = widgets.Layout(width="100px")  # Define the width of the buttons

    run_button = widgets.Button(
        description="Run", button_style="success", icon="play", layout=button_layout
    )
    save_button = widgets.Button(
        description="Save", button_style="info", icon="save", layout=button_layout
    )
    load_button = widgets.Button(
        description="Load",
        button_style="warning",
        icon="folder-open",
        layout=button_layout,
    )
    refresh_button = widgets.Button(
        description="Refresh",
        button_style="danger",
        icon="refresh",
        layout=button_layout,
    )

    output = widgets.Output(
        layout=widgets.Layout(
            width=f"{width}px",
            border="1px solid black",
            min_height="100px",
            left="6pt",
        )
    )

    def on_run_button_clicked(b):
        with output:
            output.clear_output()
            try:
                exec(editor.value)  # Using 'exec' to execute code
                display(wf.run())
            except Exception as e:
                print(f"An error occurred: {e}")

    def on_save_button_clicked(b):
        with output:
            output.clear_output()
            # Add your save functionality here...
            print("Save button clicked")

    def on_load_button_clicked(b):
        with output:
            output.clear_output()
            # Add your load functionality here...
            print("Load button clicked")

    def on_refresh_button_clicked(b):
        with output:
            output.clear_output()
            # Add your refresh functionality here...
            print("Refresh button clicked")

    run_button.on_click(on_run_button_clicked)
    save_button.on_click(on_save_button_clicked)
    load_button.on_click(on_load_button_clicked)
    refresh_button.on_click(on_refresh_button_clicked)

    toolbar_layout = widgets.Layout(
        width=f"{width}px",
        border="1px solid black",
        left="6pt",
    )
    toolbar = widgets.HBox(
        [run_button, save_button, load_button, refresh_button], layout=toolbar_layout
    )

    editor_widget = widgets.VBox(
        [toolbar, ipywidget_code_editor, output],
        layout=widgets.Layout(align_items="stretch"),
    )
    display(editor_widget)

In [35]:
code = generate_wf_code(wf)
toolbar(code, width=800)

In [ ]:
import panel
code = 'print(1)'

editor = panel.widgets.CodeEditor(
    value=code,
    language="python",
    theme="monokai",
    min_height=100,
    max_width=400,
    sizing_mode="stretch_width",
)
ipywidget_code_editor = panel.ipywidget(editor)
ipywidget_code_editor

i

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from io import BytesIO

def save_file(b):
    file_content = b"Your file content here"  # Replace with your actual file content
    file = BytesIO(file_content)
    
    widgets.FileDownload(
        file,
        filename="saved_file.txt",
        label="Download File",
        button_style="primary"
    )

save_button = widgets.Button(description="Save File")
save_button.on_click(save_file)

display(save_button)

In [ ]:
widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

In [ ]:
from collections import defaultdict

def topological_sort(nodes, edges):
    # Kahn's algorithm for topological sorting
    # Create a graph and in-degree count for each node
    graph = defaultdict(list)
    in_degree = {node: 0 for node in nodes}
    
    # Build the graph and count in-degrees
    for edge in edges:
        n_i, n_j = edge
        graph[n_i].append(n_j)
        in_degree[n_j] += 1
    
    # Initialize queue with nodes having 0 in-degree
    queue = [node for node in nodes if in_degree[node] == 0]
    
    result = []
    while queue:
        node = queue.pop(0)
        result.append(node)
        
        # Reduce in-degree of adjacent nodes
        for neighbor in graph[node]:
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                queue.append(neighbor)
    
    # Check if there's a cycle
    if len(result) != len(nodes):
        return None  # Graph has a cycle
    
    return result

In [ ]:
nodes = [1, 2, 4, -1]
edges = [(1,2), (1,4)]

In [ ]:
topological_sort(nodes, edges)

In [ ]:
def convert_graph_to_int_nodes(graph):
    

In [ ]:
graph.nodes

In [ ]:


def convert_to_integer_representation(nodes, edges):
    # Create a dictionary mapping node labels to indices
    node_to_index = {node[0]: index for index, node in enumerate(nodes)}
    
    # Convert edge list to integer representation
    for edge in edges:
        print (edge)
    # integer_edges = [(node_to_index[edge[0].split('/')[0]], node_to_index[edge[1].split('/')[0]]) for edge in edges]
    
    return integer_edges

In [ ]:
convert_to_integer_representation(graph.nodes, graph.edges)

In [ ]:
graph.nodes